In [328]:
import torch
import tiktoken

In [329]:
raw_text = "This is my dog Raylan"

In [330]:
tokenizer = tiktoken.get_encoding("gpt2")

In [331]:
enc_text = tokenizer.encode( raw_text )
print(enc_text)

[1212, 318, 616, 3290, 7760, 9620]


In [332]:
vocab_size = 5
output_dim = 8
inputs = torch.nn.Embedding( vocab_size, output_dim )
print( inputs.weight )


Parameter containing:
tensor([[-1.0906, -0.8287, -1.9239, -2.2434, -0.5622,  0.5270,  0.8148,  1.0701],
        [ 0.3235, -0.6881, -0.9939,  0.0727, -0.9362,  0.3649, -0.9449,  0.3256],
        [-0.8313, -0.5895,  0.5404, -0.2123,  0.6798,  0.8398,  0.3375, -1.0469],
        [ 1.5933, -1.0042, -1.9855, -0.1813,  1.5931, -0.5966, -0.5984,  0.5162],
        [ 0.4964, -0.2128, -1.0523,  1.2179,  1.3719, -0.0365,  0.0908,  0.0134]],
       requires_grad=True)


In [333]:
inputs = inputs.weight.data
print(inputs)


tensor([[-1.0906, -0.8287, -1.9239, -2.2434, -0.5622,  0.5270,  0.8148,  1.0701],
        [ 0.3235, -0.6881, -0.9939,  0.0727, -0.9362,  0.3649, -0.9449,  0.3256],
        [-0.8313, -0.5895,  0.5404, -0.2123,  0.6798,  0.8398,  0.3375, -1.0469],
        [ 1.5933, -1.0042, -1.9855, -0.1813,  1.5931, -0.5966, -0.5984,  0.5162],
        [ 0.4964, -0.2128, -1.0523,  1.2179,  1.3719, -0.0365,  0.0908,  0.0134]])


In [334]:
inputs.shape

torch.Size([5, 8])

In [335]:
for row in inputs:
    print(row.tolist())

[-1.0905534029006958, -0.8287144899368286, -1.9239084720611572, -2.2434046268463135, -0.5622209310531616, 0.5270102024078369, 0.8147833943367004, 1.0700957775115967]
[0.3235299289226532, -0.6881058216094971, -0.9938681721687317, 0.07266741245985031, -0.9361704587936401, 0.36492493748664856, -0.9448953866958618, 0.3256247341632843]
[-0.8312854170799255, -0.5895064473152161, 0.5404170155525208, -0.2123185694217682, 0.6798174977302551, 0.8398153781890869, 0.33749473094940186, -1.0468982458114624]
[1.5933021306991577, -1.0041815042495728, -1.9854753017425537, -0.18129108846187592, 1.5930616855621338, -0.5966479778289795, -0.5983661413192749, 0.5161851644515991]
[0.49640342593193054, -0.21282871067523956, -1.0523451566696167, 1.2178808450698853, 1.3718595504760742, -0.03649982437491417, 0.09077786654233932, 0.013367624022066593]


In [336]:
x = torch.Tensor([1.1,2.3])
y = torch.Tensor([3.4,-2.1])

In [337]:
1.1*3.4 + 2.3*-2.1

-1.0899999999999999

In [338]:
torch.dot(x,y)

tensor(-1.0900)

In [339]:
query = inputs[2]
print(query)

tensor([-0.8313, -0.5895,  0.5404, -0.2123,  0.6798,  0.8398,  0.3375, -1.0469])


In [340]:
for i in range( len( inputs )):
    print( torch.dot(query, inputs[i]))

tensor(0.0468)
tensor(-1.4056)
tensor(3.7530)
tensor(-1.9274)
tensor(-0.1959)


In [341]:
attention_scores_2 = torch.zeros(len(inputs))
for i in range( len( inputs )):
    attention_scores_2[i] = torch.dot(query, inputs[i])
print(attention_scores_2)

tensor([ 0.0468, -1.4056,  3.7530, -1.9274, -0.1959])


In [342]:
# #normalize the attention scores using the softmax function
# def softmax(x):
#     torch.exp(x) / torch.exp(x).sum()

In [343]:
attention_weights_2 = torch.softmax(attention_scores_2, dim = 0)
print(attention_weights_2)
print( attention_weights_2.sum() )

tensor([0.0233, 0.0055, 0.9497, 0.0032, 0.0183])
tensor(1.)


In [345]:
context_vector_2 = torch.zeros( query.shape )
for i in range(len(attention_weights_2)):
    context_vector_2 += attention_weights_2[i]*inputs[i]
print(context_vector_2)

tensor([-0.7989, -0.5901,  0.4372, -0.2319,  0.6576,  0.8092,  0.3341, -0.9655])


In [348]:
# get all of the attention scores by matrix multiplication
attention_scores = inputs @ inputs.T
attention_scores

tensor([[13.0132,  2.2637,  0.0468,  2.1759, -1.7748],
        [ 2.2637,  3.5797, -1.4056,  2.1910,  0.0624],
        [ 0.0468, -1.4056,  3.7530, -1.9274, -0.1959],
        [ 2.1759,  2.1910, -1.9274, 11.0403,  5.0331],
        [-1.7748,  0.0624, -0.1959,  5.0331,  4.7741]])

In [350]:
attention_weights = torch.softmax( attention_scores, dim = -1)
attention_weights

tensor([[9.9996e-01, 2.1456e-05, 2.3375e-06, 1.9653e-05, 3.7813e-07],
        [1.7259e-01, 6.4344e-01, 4.3999e-03, 1.6047e-01, 1.9098e-02],
        [2.3333e-02, 5.4603e-03, 9.4966e-01, 3.2402e-03, 1.8306e-02],
        [1.4095e-04, 1.4308e-04, 2.3281e-06, 9.9726e-01, 2.4542e-03],
        [6.1897e-04, 3.8865e-03, 3.0019e-03, 5.6014e-01, 4.3235e-01]])

In [351]:
attention_weights.sum(axis = -1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [353]:
context_vectors = attention_weights @ inputs
context_vectors

tensor([[-1.0905, -0.8287, -1.9239, -2.2433, -0.5622,  0.5270,  0.8147,  1.0701],
        [ 0.2815, -0.7536, -1.3079, -0.3472, -0.4146,  0.2330, -0.5602,  0.4727],
        [-0.7989, -0.5901,  0.4372, -0.2319,  0.6576,  0.8092,  0.3341, -0.9655],
        [ 1.5900, -1.0022, -1.9830, -0.1781,  1.5919, -0.5950, -0.5965,  0.5150],
        [ 1.1052, -0.6595, -1.5706,  0.4233,  1.4835, -0.3457, -0.2981,  0.2937]])